In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/all-minilm-l6-v2-tuning-model-add/__results__.html
/kaggle/input/all-minilm-l6-v2-tuning-model-add/submission.csv
/kaggle/input/all-minilm-l6-v2-tuning-model-add/submission_2.csv
/kaggle/input/all-minilm-l6-v2-tuning-model-add/__resultx__.html
/kaggle/input/all-minilm-l6-v2-tuning-model-add/__notebook__.ipynb
/kaggle/input/all-minilm-l6-v2-tuning-model-add/__output__.json
/kaggle/input/all-minilm-l6-v2-tuning-model-add/submission_1.csv
/kaggle/input/all-minilm-l6-v2-tuning-model-add/custom.css
/kaggle/input/39-50-gpt-oss-120b-tir-dynamictime-out-pooling/reference.csv
/kaggle/input/39-50-gpt-oss-120b-tir-dynamictime-out-pooling/submission.parquet
/kaggle/input/39-50-gpt-oss-120b-tir-dynamictime-out-pooling/__results__.html
/kaggle/input/39-50-gpt-oss-120b-tir-dynamictime-out-pooling/vllm_server.log
/kaggle/input/39-50-gpt-oss-120b-tir-dynamictime-out-pooling/__notebook__.ipynb
/kaggle/input/39-50-gpt-oss-120b-tir-dynamictime-out-pooling/__output__.json
/kaggle/input/39-50-

In [2]:
# ========== SUPPRESS ALL WARNINGS ==========
import warnings
import os
import sys

warnings.filterwarnings("ignore")
os.environ["PYTHONWARNINGS"] = "ignore"
os.environ["PYDEVD_DISABLE_FILE_VALIDATION"] = "1"
os.environ["TRANSFORMERS_VERBOSITY"] = "error"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

warnings.filterwarnings("ignore", category=SyntaxWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

import logging
logging.basicConfig(level=logging.ERROR)
logging.getLogger("pydevd").setLevel(logging.ERROR)

# ========== AI MATHEMATICAL OLYMPIAD - HARD-CODED MATHEMATICAL SOLVER ==========
# NO WANDB - Pure hard-coded mathematical reasoning
# Transformer only used as last resort reference

import os, sys, json, re, hashlib, time, multiprocessing, gc
import numpy as np
import pandas as pd
import polars as pl
import torch
import sympy
from sympy import symbols, solve, simplify, expand, factor, Rational, Integer
from sympy.parsing.sympy_parser import parse_expr
from sklearn.model_selection import KFold
import lightgbm as lgb

# ========== SETUP & PATHS ==========
sys.path.append('/kaggle/input/ai-mathematical-olympiad-progress-prize-3')
import kaggle_evaluation.aimo_3_inference_server

MODEL_PATH = "/kaggle/input/all-minilm-l6-v2-tuning-model-add"
REF_DATA_PATH = "/kaggle/input/ai-mathematical-olympiad-progress-prize-3/reference.csv"
TEST_DATA_PATH = "/kaggle/input/ai-mathematical-olympiad-progress-prize-3/test.csv"
SOLVER_TIMEOUT = 25 

# ========== METHOD 1: MODULO ARITHMETIC SOLVER ==========
def solve_modulo(text):
    """HARD-CODED: Solves modulo arithmetic problems"""
    patterns = [
        r'(\d+)\s*\\pmod\s*{\s*(\d+)\s*}',
        r'(\d+)\s*mod\s+(\d+)',
        r'(\d+)\s*%\s*(\d+)',
        r'(\d+)\s*modulo\s+(\d+)',
        r'find\s+(\d+)\s*mod\s+(\d+)',
        r'(\d+)\s*≡\s*(\d+)\s*\(mod\s+(\d+)\)',
    ]
    
    for pattern in patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            if len(match.groups()) == 2:
                a, b = int(match.group(1)), int(match.group(2))
                result = a % b
                return result % 10, "modulo_arithmetic"
            elif len(match.groups()) == 3:
                a, b, n = int(match.group(1)), int(match.group(2)), int(match.group(3))
                result = (a - b) % n
                return result % 10, "modulo_congruence"
    
    return None, None

# ========== METHOD 2: ALGEBRAIC EQUATION SOLVER ==========
def solve_algebraic(text):
    """HARD-CODED: Solves algebraic equations"""
    try:
        latex_matches = re.findall(r'\$(.*?)\$', text)
        for expr_str in latex_matches:
            try:
                expr_str = expr_str.replace('^', '**').replace('\\', '')
                if '=' in expr_str:
                    left, right = expr_str.split('=', 1)
                    left_expr = parse_expr(left.strip())
                    right_expr = parse_expr(right.strip())
                    equation = left_expr - right_expr
                else:
                    equation = parse_expr(expr_str)
                
                vars_in_expr = equation.free_symbols
                if vars_in_expr:
                    x = list(vars_in_expr)[0]
                    solutions = solve(equation, x)
                    if solutions:
                        result = int(abs(float(solutions[0].evalf()))) % 10
                        return result, "algebraic_equation"
            except:
                continue
        
        equation_patterns = [
            r'solve\s+for\s+\w+:\s*([^\.]+)',
            r'(\w+)\s*[+\-*/]\s*\d+\s*=\s*\d+',
            r'\d+\s*[+\-*/]\s*(\w+)\s*=\s*\d+',
        ]
        
        for pattern in equation_patterns:
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                eq_str = match.group(1) if match.groups() else match.group(0)
                numbers = re.findall(r'\d+', eq_str)
                if len(numbers) >= 2:
                    coefs = [int(n) for n in numbers]
                    if len(coefs) >= 2:
                        result = coefs[-1] % 10
                        return result, "algebraic_linear"
    except:
        pass
    
    return None, None

# ========== METHOD 3: ARITHMETIC EXPRESSION EVALUATOR ==========
def solve_arithmetic(text):
    """HARD-CODED: Evaluates arithmetic expressions"""
    try:
        patterns = [
            r'what\s+is\s+([\d\+\-\*\/\(\)\s]+)\??',
            r'calculate\s+([\d\+\-\*\/\(\)\s]+)',
            r'compute\s+([\d\+\-\*\/\(\)\s]+)',
            r'evaluate\s+([\d\+\-\*\/\(\)\s]+)',
            r'(\d+\s*[\+\-\*\/]\s*\d+)',
        ]
        
        for pattern in patterns:
            matches = re.findall(pattern, text, re.IGNORECASE)
            for expr_str in matches:
                try:
                    expr_str = expr_str.strip()
                    result = eval(expr_str.replace('^', '**'))
                    return int(abs(result)) % 10, "arithmetic_evaluation"
                except:
                    continue
    except:
        pass
    
    return None, None

# ========== METHOD 4: NUMBER THEORY SOLVER ==========
def solve_number_theory(text):
    """HARD-CODED: Solves number theory problems"""
    try:
        gcd_match = re.search(r'gcd\((\d+),\s*(\d+)\)|greatest\s+common\s+divisor\s+of\s+(\d+)\s+and\s+(\d+)', text, re.IGNORECASE)
        if gcd_match:
            groups = gcd_match.groups()
            a = int(groups[0] or groups[2])
            b = int(groups[1] or groups[3])
            result = np.gcd(a, b) % 10
            return result, "number_theory_gcd"
        
        lcm_match = re.search(r'lcm\((\d+),\s*(\d+)\)|least\s+common\s+multiple\s+of\s+(\d+)\s+and\s+(\d+)', text, re.IGNORECASE)
        if lcm_match:
            groups = lcm_match.groups()
            a = int(groups[0] or groups[2])
            b = int(groups[1] or groups[3])
            result = np.lcm(a, b) % 10
            return result, "number_theory_lcm"
        
        fact_match = re.search(r'(\d+)!|factorial\s+of\s+(\d+)', text, re.IGNORECASE)
        if fact_match:
            n = int(fact_match.group(1) or fact_match.group(2))
            if n <= 10:
                result = 1
                for i in range(1, n + 1):
                    result *= i
                return result % 10, "number_theory_factorial"
        
        digit_sum_match = re.search(r'sum\s+of\s+digits\s+of\s+(\d+)|digit\s+sum\s+of\s+(\d+)', text, re.IGNORECASE)
        if digit_sum_match:
            num = int(digit_sum_match.group(1) or digit_sum_match.group(2))
            result = sum(int(d) for d in str(num)) % 10
            return result, "number_theory_digit_sum"
        
        prime_match = re.search(r'is\s+(\d+)\s+prime|prime\s+number\s+(\d+)', text, re.IGNORECASE)
        if prime_match:
            n = int(prime_match.group(1) or prime_match.group(2))
            if n > 1:
                is_prime = all(n % i != 0 for i in range(2, int(n**0.5) + 1))
                return (1 if is_prime else 0), "number_theory_prime"
    except:
        pass
    
    return None, None

# ========== METHOD 5: GEOMETRY SOLVER ==========
def solve_geometry(text):
    """HARD-CODED: Solves basic geometry problems"""
    try:
        text_lower = text.lower()
        
        area_patterns = [
            (r'area\s+of\s+(?:a\s+)?(?:square|rectangle)\s+with\s+side\s+(\d+)', lambda m: int(m.group(1))**2),
            (r'area\s+of\s+(?:a\s+)?circle\s+with\s+radius\s+(\d+)', lambda m: int(3.14159 * int(m.group(1))**2)),
            (r'area\s+=\s+(\d+)\s*\*\s*(\d+)', lambda m: int(m.group(1)) * int(m.group(2))),
        ]
        
        for pattern, func in area_patterns:
            match = re.search(pattern, text_lower)
            if match:
                result = func(match) % 10
                return result, "geometry_area"
        
        peri_match = re.search(r'perimeter\s+of\s+(?:a\s+)?(?:square|rectangle)\s+with\s+side\s+(\d+)', text_lower)
        if peri_match:
            side = int(peri_match.group(1))
            result = (4 * side) % 10
            return result, "geometry_perimeter"
        
        angle_match = re.search(r'angle\s+of\s+(\d+)\s+degrees|(\d+)\s+degrees', text_lower)
        if angle_match:
            angle = int(angle_match.group(1) or angle_match.group(2))
            if 'complement' in text_lower:
                result = (90 - angle) % 10
            elif 'supplement' in text_lower:
                result = (180 - angle) % 10
            else:
                result = angle % 10
            return result, "geometry_angle"
    except:
        pass
    
    return None, None

# ========== METHOD 6: SEQUENCE/PATTERN SOLVER ==========
def solve_sequence(text):
    """HARD-CODED: Solves sequence and pattern problems"""
    try:
        seq_match = re.search(r'(\d+(?:\s*,\s*\d+)+)\s*\??', text)
        if seq_match:
            seq_str = seq_match.group(1)
            numbers = [int(n.strip()) for n in seq_str.split(',')]
            
            if len(numbers) >= 3:
                diffs = [numbers[i+1] - numbers[i] for i in range(len(numbers)-1)]
                if len(set(diffs)) == 1:
                    next_num = numbers[-1] + diffs[0]
                    return next_num % 10, "sequence_arithmetic"
                
                if all(numbers[i] != 0 for i in range(len(numbers)-1)):
                    ratios = [numbers[i+1] / numbers[i] for i in range(len(numbers)-1)]
                    if len(set(ratios)) == 1:
                        next_num = int(numbers[-1] * ratios[0])
                        return next_num % 10, "sequence_geometric"
                
                if len(numbers) >= 3 and numbers[2] == numbers[0] + numbers[1]:
                    next_num = numbers[-1] + numbers[-2]
                    return next_num % 10, "sequence_fibonacci"
    except:
        pass
    
    return None, None

# ========== METHOD 7: COMBINATORICS SOLVER ==========
def solve_combinatorics(text):
    """HARD-CODED: Solves combinatorics problems"""
    try:
        text_lower = text.lower()
        
        comb_match = re.search(r'C\((\d+),\s*(\d+)\)|(\d+)\s+choose\s+(\d+)', text_lower)
        if comb_match:
            groups = comb_match.groups()
            n = int(groups[0] or groups[2])
            k = int(groups[1] or groups[3])
            if 0 <= k <= n and n <= 20:
                from math import comb
                result = comb(n, k) % 10
                return result, "combinatorics_combination"
        
        perm_match = re.search(r'P\((\d+),\s*(\d+)\)|(\d+)\s+permute\s+(\d+)', text_lower)
        if perm_match:
            groups = perm_match.groups()
            n = int(groups[0] or groups[2])
            k = int(groups[1] or groups[3])
            if 0 <= k <= n and n <= 10:
                from math import perm
                result = perm(n, k) % 10
                return result, "combinatorics_permutation"
    except:
        pass
    
    return None, None

# ========== METHOD 8: HEURISTIC SOLVER ==========
def solve_heuristic(text):
    """HARD-CODED: Heuristic methods"""
    try:
        numbers = [int(n) for n in re.findall(r'\d+', text)]
        
        if len(numbers) >= 2:
            if 'sum' in text.lower() or 'add' in text.lower():
                return sum(numbers) % 10, "heuristic_sum"
            
            if 'product' in text.lower() or 'multiply' in text.lower():
                return np.prod(numbers) % 10, "heuristic_product"
            
            if 'difference' in text.lower() or 'subtract' in text.lower():
                return abs(numbers[0] - numbers[-1]) % 10, "heuristic_difference"
            
            if 'average' in text.lower() or 'mean' in text.lower():
                return int(np.mean(numbers)) % 10, "heuristic_average"
            
            return max(numbers) % 10, "heuristic_max"
        
        elif len(numbers) == 1:
            return numbers[0] % 10, "heuristic_single_digit"
    except:
        pass
    
    return None, None

# ========== METHOD 9: TRANSFORMER FALLBACK (LAST RESORT) ==========
def get_transformer_embedding(text):
    """TRANSFORMER REFERENCE: Only used if all hard-coded methods fail"""
    try:
        from transformers import AutoTokenizer, AutoModel, AutoConfig
        
        device = "cuda" if torch.cuda.is_available() else "cpu"
        config = AutoConfig.from_pretrained(MODEL_PATH)
        if not hasattr(config, 'model_type'): 
            config.model_type = "bert"
        
        tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
        model = AutoModel.from_pretrained(
            MODEL_PATH, 
            config=config,
            torch_dtype=torch.float16 if device == "cuda" else torch.float32
        ).to(device)
        model.eval()
        
        inputs = tokenizer([text], return_tensors="pt", padding=True, truncation=True).to(device)
        with torch.no_grad():
            out = model(**inputs, output_hidden_states=True)
            embedding = out.hidden_states[-1].mean(dim=1).cpu().float().numpy()[0, :16]
        
        return embedding
    except:
        return np.zeros(16)

# ========== MAIN SOLVER: EXHAUSTS ALL METHODS ==========
MEMO = {}
LGB_MODELS = []

def solve_with_all_methods(problem_id, problem_text):
    """
    EXHAUSTS ALL MATHEMATICAL REASONING METHODS IN ORDER:
    1. Modulo arithmetic
    2. Algebraic equations
    3. Arithmetic evaluation
    4. Number theory
    5. Geometry
    6. Sequences/patterns
    7. Combinatorics
    8. Heuristic methods
    9. ML fallback (only if all above fail)
    """
    # Check cache
    h = hashlib.md5(problem_text.encode()).hexdigest()
    if h in MEMO:
        return MEMO[h][0], MEMO[h][1]
    
    # Try all hard-coded methods in order
    methods = [
        solve_modulo,
        solve_algebraic,
        solve_arithmetic,
        solve_number_theory,
        solve_geometry,
        solve_sequence,
        solve_combinatorics,
        solve_heuristic,
    ]
    
    for method in methods:
        ans, method_name = method(problem_text)
        if ans is not None:
            reasoning = f"Hard-coded: {method_name}"
            MEMO[h] = (ans, reasoning)
            return ans, reasoning
    
    # LAST RESORT: ML fallback
    if LGB_MODELS:
        try:
            embedding = get_transformer_embedding(problem_text)
            numbers = [int(n) for n in re.findall(r'\d+', problem_text)]
            features = np.concatenate([
                embedding,
                [len(problem_text.split()), len(numbers), sum(numbers) % 10]
            ]).reshape(1, -1)
            
            preds = np.mean([m.predict(features) for m in LGB_MODELS], axis=0)
            ans = int(np.clip(np.round(preds[0]), 0, 9))
            reasoning = f"ML fallback: All hard-coded methods failed"
            MEMO[h] = (ans, reasoning)
            return ans, reasoning
        except:
            pass
    
    # Ultimate fallback
    ans = hash(problem_text) % 10
    reasoning = "Ultimate fallback: hash-based"
    MEMO[h] = (ans, reasoning)
    return ans, reasoning

# ========== TRAIN ML MODELS (ONLY FOR FALLBACK) ==========
def train_ml_fallback():
    """Train ML models only as last resort fallback"""
    if not os.path.exists(REF_DATA_PATH):
        return
    
    try:
        df = pd.read_csv(REF_DATA_PATH)
        all_feats = []
        all_answers = []
        
        for idx, row in df.iterrows():
            txt = row['problem']
            answer = int(row['answer']) % 10
            
            embedding = get_transformer_embedding(txt)
            numbers = [int(n) for n in re.findall(r'\d+', txt)]
            features = np.concatenate([
                embedding,
                [len(txt.split()), len(numbers), sum(numbers) % 10]
            ])
            all_feats.append(features)
            all_answers.append(answer)
        
        X = np.array(all_feats)
        y = np.array(all_answers)
        
        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        for t_idx, v_idx in kf.split(X):
            m = lgb.LGBMRegressor(n_estimators=30, learning_rate=0.1, verbose=-1)
            m.fit(X[t_idx], y[t_idx])
            LGB_MODELS.append(m)
        
        print(f"✅ Trained {len(LGB_MODELS)} ML fallback models")
    except Exception as e:
        print(f"⚠️ ML training failed: {e}")

# ========== PREDICTION API ==========
def predict(id_series: pl.Series, prob_series: pl.Series) -> pl.DataFrame:
    """Main prediction function"""
    pid = id_series.item(0)
    prob_text = prob_series.item(0)
    
    gc.collect()
    if torch.cuda.is_available(): 
        torch.cuda.empty_cache()
    
    ans, reasoning = solve_with_all_methods(pid, prob_text)
    return pl.DataFrame({'id': [pid], 'answer': [ans]})

# ========== MAIN EXECUTION ==========
if __name__ == "__main__":
    print("=" * 80)
    print("HARD-CODED MATHEMATICAL SOLVER - NO WANDB")
    print("=" * 80)
    
    # Train ML fallback (optional)
    print("\n[OPTIONAL] Training ML fallback models...")
    train_ml_fallback()
    
    # Load test questions
    print("\n[STEP 1] Loading test questions...")
    if os.path.exists(TEST_DATA_PATH):
        test_df = pd.read_csv(TEST_DATA_PATH)
        print(f"✅ Loaded {len(test_df)} test questions")
        
        print("\n[STEP 2] Solving with hard-coded methods...")
        predictions = []
        
        for idx, row in test_df.iterrows():
            pid = row['id']
            prob_text = row['problem'] if 'problem' in row else str(row)
            
            ans, reasoning = solve_with_all_methods(pid, prob_text)
            predictions.append({
                'id': pid,
                'answer': ans,
                'reasoning': reasoning
            })
            
            if (idx + 1) % 10 == 0:
                print(f"  Processed {idx + 1}/{len(test_df)} questions")
        
        # Create submission.csv
        print("\n[STEP 3] Creating submission.csv...")
        submission_df = pd.DataFrame({
            'id': [p['id'] for p in predictions],
            'answer': [p['answer'] for p in predictions]
        })
        
        submission_df['answer'] = submission_df['answer'].clip(0, 9).astype(int)
        submission_df.to_csv("submission.csv", index=False)
        print(f"✅ Created submission.csv with {len(submission_df)} predictions")
        
        # Method statistics
        method_counts = {}
        for p in predictions:
            method = p['reasoning'].split(':')[0]
            method_counts[method] = method_counts.get(method, 0) + 1
        
        print("\n[STEP 4] Method usage statistics:")
        for method, count in sorted(method_counts.items(), key=lambda x: -x[1]):
            print(f"   {method}: {count} problems")
    
    # Start inference server
    print("\n[STEP 5] Starting inference server...")
    server = kaggle_evaluation.aimo_3_inference_server.AIMO3InferenceServer(predict)
    
    try:
        server.serve()
    except KeyboardInterrupt:
        print("\n⚠️ Server stopped")
    finally:
        print(f"\n✅ Cached {len(MEMO)} solutions")
        print("✅ Submission file ready: submission.csv")

HARD-CODED MATHEMATICAL SOLVER - NO WANDB

[OPTIONAL] Training ML fallback models...
✅ Trained 5 ML fallback models

[STEP 1] Loading test questions...
✅ Loaded 3 test questions

[STEP 2] Solving with hard-coded methods...

[STEP 3] Creating submission.csv...
✅ Created submission.csv with 3 predictions

[STEP 4] Method usage statistics:
   Hard-coded: 3 problems

[STEP 5] Starting inference server...

✅ Cached 3 solutions
✅ Submission file ready: submission.csv
